In [ ]:
import numpy as np
from scipy.stats import truncnorm
from numpy.linalg import eigvals
import pandas as pd
#---------------------------------------------------------------------------
# Parameters for fidelity distribution (truncated Gaussian)
mu = 0.95
sigma = 0.05
lower, upper = 0.5, 1.0
zeta = {1: 0.85, 2: 0.90, 3: 0.95}  # Fidelity thresholds

m_values = [1, 2, 3]  # Now include level 1 for primitive edges
p_memory = 1          # Node survival probability
q = 0.96              # Success probability for edge creation or GHZ measurement
d = 4                 # Node degree in level m=2 (e.g., grid)
#---------------------------------------------------------------------------
# Truncated Gaussian distribution for fidelity
a, b = (lower - mu) / sigma, (upper - mu) / sigma
fidelity_dist = truncnorm(a, b, loc=mu, scale=sigma)

# Compute p_fid = P(F > zeta[m]) for each m
p_fid_dict = {m: 1 - fidelity_dist.cdf(zeta[m]) for m in m_values}

# Define p_H for each level
p_H_dict = {
    m: (q * p_fid_dict[m]) if m == 1 else (q**(m + 1) * (p_fid_dict[m]**m))
    for m in m_values
}
#---------------------------------------------------------------------------
# Simulate degree distributions: binomial for simplicity
max_k = d
P_qm = {}

for m in m_values:
    if m == 1:
        p_1GHZ = q * p_fid_dict[m]  # For primitive edges, simpler logic
    else:
        p_1GHZ = q**m * (p_fid_dict[m]**m) * np.math.comb((d - 1), (m - 1))  # Include combinatorics for GHZ fusion
    probs = [np.math.comb(d, k) * (p_1GHZ)**k * (1 - p_1GHZ)**(d - k) for k in range(max_k + 1)]
    probs = np.array(probs)
    probs /= probs.sum()  # Normalize
    P_qm[m] = probs
#---------------------------------------------------------------------------
# Compute expectations: <q_m>, <q_m q_m'>, etc.
E_q = {m: sum(k * P_qm[m][k] for k in range(max_k + 1)) for m in m_values}
E_q_q = {(m, mp): sum(k * kp * P_qm[m][k] * P_qm[mp][kp]
                      for k in range(max_k + 1) for kp in range(max_k + 1))
         for m in m_values for mp in m_values}

# Build Jacobian matrix G
G = np.zeros((len(m_values), len(m_values)))
for i, m in enumerate(m_values):
    for j, mp in enumerate(m_values):
        numerator = E_q_q[(m, mp)] - (1 if m == mp else 0) * E_q[m]
        G[i, j] = p_H_dict[mp] * p_memory**(mp - 1) * (mp - 1) * numerator / E_q[m]
#---------------------------------------------------------------------------
# Compute largest eigenvalue
lambda_max = max(eigvals(G)).real

# Display Jacobian matrix
import ace_tools as tools
tools.display_dataframe_to_user(name="Jacobian Matrix G", dataframe=pd.DataFrame(G, index=m_values, columns=m_values))

lambda_max